In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs


import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
x0,x1,y0,y1 = 3445,3755,7730,8065

In [ ]:
ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

In [ ]:
cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.algae_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.deep'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

lon0,lon1 = 245.8,251
lat0,lat1 = -75.2,-74.4

proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)


In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.01,right=.9,top=.9,bottom=.05,wspace=.01,hspace=.01)

fig = plt.figure(figsize=(7,3))

ax = fig.add_subplot(121)
ax.set_axis_off()
ax.set_title('a) LADDIE model',loc='left')

ax = fig.add_subplot(122,projection=proj)

im1 = ax.pcolormesh(ds.lon,ds.lat,ds.bed,cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==0,np.nan,ds.thickness),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=0,vmax=2000)
var = np.where(ds.mask==0,np.nan,ds.bed)
var = np.where(ds.mask==3,np.nan,var)
im3 = ax.pcolormesh(ds.lon,ds.lat,var,cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

cb = plt.colorbar(im2,shrink=.92)#,cax=ax)
cb.set_label('Ice shelf thickness [m]')

ax.set_xticks([])
ax.set_yticks([])
ax.set_axis_off()
ax.set_title('b) Crosson-Dotson geography',loc='left')

ax.text(.78,.22,'Kohler',rotation=0,transform=ax.transAxes,c='w')
ax.text(.2,.15,'Dotson Channel',rotation=30,transform=ax.transAxes)
ax.text(.1,.56,'Bear Peninsula',transform=ax.transAxes,c='w')
ax.text(.75,.25,'Smith\nWest',rotation=-45,transform=ax.transAxes,c='w')
ax.text(.8,.47,'Smith\nEast',rotation=0,transform=ax.transAxes,c='w')
ax.text(.75,.63,'Pope',rotation=0,transform=ax.transAxes,c='w')
ax.text(.4,.72,'Crosson\nice shelf',rotation=0,transform=ax.transAxes)
ax.text(.2,.3,'Dotson\nice shelf',rotation=0,transform=ax.transAxes)





axins = inset_axes(ax,width="32%",height="32%",loc='upper right',borderpad=1,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))
axins.set_extent([-180, 180, -90, -65], crs=ccrs.PlateCarree())
axins.coastlines(linewidth=0.2, zorder=2, resolution='10m',color='k')
axins.scatter((lon0+lon1)/2,(lat0+lat1)/2,10,facecolors='r',marker='s',edgecolors='none',transform=ccrs.PlateCarree())
axins.set_axis_off()

plt.savefig('../../figures/draftplot_overview.png',dpi=450)